<a href="https://colab.research.google.com/github/filipediniz/Coursera_Capstone/blob/master/Week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests_html
!pip install pixiedust
!pip install geocoder
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

     |████████████████████████████████| 1.2MB 6.5MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=5702c9607641de74a74dc57d00e862f22415efb604f6a30b34fbb5f50c3c39bc
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for pyppeteer: filename=pyppeteer-0.0.25-cp36-none-any.whl size=78360 sha256=d5dd8cd4fd8a266d1af42d0ca2f60d46d845ac5b3bc9e9a65fe10f94c3c8edd5
  Stored in directory: /root/.cache/pip/wheels/34/e0/5d/070e22eceecf7ecd5fa4b86bbc18c1c7d0b90e02e9b57f35eb
  Created wheel for parse: filename=parse-1.15.0-cp36-none-any.whl size=23710 sha256=2937050015997e1d0df2da1dc95234e4eb6d4405a52174efdf68953c680a0c21
  Stored in directory: /root/.cache/pip/wheels/07/ee/c8/eced0759f09fc635398ab1b8e89c38549b28e5db7fd4a53ba5
Successfully built fake-useragent pyppeteer parse
     |████████████████████████████████| 204k

In [2]:
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import numpy as np
import matplotlib.pyplot as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import pixiedust
import requests
from requests_html import HTMLSession
import re as regex

import datetime as dt
from pandas_datareader import data

from IPython.display import clear_output

import pickle

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


1. Start by creating a new Notebook for this assignment.

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [0]:
#%%pixie_debugger

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')    
try:
    DF = pd.read_html(ret.text, decimal=",", thousands='.', encoding='utf-8')[0]
    #DF.columns = DF.iloc[0]
    #DF = DF.reset_index()
except:
    print('Página sem dados')
    

In [0]:
from bs4 import BeautifulSoup

session = HTMLSession()
DF = pd.DataFrame()
ret = session.get('https://en.wikipedia.org/wiki/Irvine,_California')    
try:    
    soup = BeautifulSoup(ret.text, 'html.parser')
except:
    print('Página sem dados')

In [69]:
html_objs = soup.find_all("h4")
villages = ''
for html_obj in html_objs:
  print(html_obj.text)
  if html_obj.text == 'Villages':    
    villages = html_obj.next_element.next_element.next_element.next_element.next_element.next_element.next_element
    break
neighborhoods = []
for village in list((list(villages.children)[1]).children):
  if village.name != None:    
    neighborhoods.append(village.text.split(' (')[0].split('[')[0])    

Villages


 3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [68]:
DF = pd.DataFrame()
DF['Neighborhood'] = neighborhoods
DF

,Neighborhood
0,El Camino Glen
1,College Park
2,The Colony
3,Columbus Grove
4,Cypress Village
5,Deerfield
6,East Irvine
7,El Camino Real
8,Greentree
9,Irvine Groves


4. Submit a link to your Notebook on your Github repository. (10 marks)

# END OF PART 1

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

In [73]:
#@title
import geocoder # import geocoder
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

for r in range(0, DF.shape[0]):
  print('Current row: {}'.format(r))
  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  g = None
  while(type(g) == type(None)):    
    print('Attempting to retrieve coordinates for address {}...'.format('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood'])))
    geolocator = Nominatim(user_agent="irvine_explorer")
    #g = geolocator.geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )          
    g = do_geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )            
    if (type(g) != type(None)):    
      print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
      DF.iloc[r]['Latitude'] = g.latitude
      DF.iloc[r]['Longitude'] = g.longitude    

Current row: 0
Attempting to retrieve coordinates for address El Camino Glen, Irvine, California...
Latitude = 33.6960782; longitude = -117.790868
Current row: 1
Attempting to retrieve coordinates for address College Park, Irvine, California...
Latitude = 33.7049508; longitude = -117.79327516984964
Current row: 2
Attempting to retrieve coordinates for address The Colony, Irvine, California...
Latitude = 33.75507555; longitude = -117.80374469402437
Current row: 3
Attempting to retrieve coordinates for address Columbus Grove, Irvine, California...
Latitude = 33.7045487; longitude = -117.7637001
Current row: 4
Attempting to retrieve coordinates for address Cypress Village, Irvine, California...
Latitude = 33.6835428; longitude = -117.75398984807549
Current row: 5
Attempting to retrieve coordinates for address Deerfield, Irvine, California...
Latitude = 33.6929179; longitude = -117.7907331
Current row: 6
Attempting to retrieve coordinates for address East Irvine, Irvine, California...
Lati

GeocoderServiceError: ignored

In [75]:
#@title
import geocoder # import geocoder
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

def do_geocode(address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

for r in range(0, DF.shape[0]):
  print('Current row: {}'.format(r))
  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  g = None
  print('Attempting to retrieve coordinates for address {}...'.format('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood'])))
  geolocator = Nominatim(user_agent="irvine_explorer")
  g = geolocator.geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )          
  #g = do_geocode('{}, Irvine, California'.format(DF.iloc[r]['Neighborhood']), )            
  if (type(g) != type(None)):    
    print('Latitude = {}; longitude = {}'.format(g.latitude, g.longitude))  
    DF.iloc[r]['Latitude'] = g.latitude
    DF.iloc[r]['Longitude'] = g.longitude    

Current row: 0
Attempting to retrieve coordinates for address El Camino Glen, Irvine, California...
Latitude = 33.6960782; longitude = -117.790868
Current row: 1
Attempting to retrieve coordinates for address College Park, Irvine, California...
Latitude = 33.7049508; longitude = -117.79327516984964
Current row: 2
Attempting to retrieve coordinates for address The Colony, Irvine, California...
Latitude = 33.75507555; longitude = -117.80374469402437
Current row: 3
Attempting to retrieve coordinates for address Columbus Grove, Irvine, California...
Latitude = 33.7045487; longitude = -117.7637001
Current row: 4
Attempting to retrieve coordinates for address Cypress Village, Irvine, California...
Latitude = 33.6835428; longitude = -117.75398984807549
Current row: 5
Attempting to retrieve coordinates for address Deerfield, Irvine, California...
Latitude = 33.6929179; longitude = -117.7907331
Current row: 6
Attempting to retrieve coordinates for address East Irvine, Irvine, California...
Lati

In [0]:
#LL = pd.read_csv('http://cocl.us/Geospatial_data')

In [0]:
'''
for i in range(0, DF.shape[0]):
  for j in range(0, LL.shape[0]):
    if DF.iloc[i]['Postcode'] == LL.iloc[j]['Postal Code']:
      print('Postal code {} coordinates are: latitude = {}; longitude = {}'.format(LL.iloc[j]['Postal Code'], LL.iloc[j]['Latitude'], LL.iloc[j]['Longitude']))
      DF.ix[i, 'Latitude'] = LL.iloc[j]['Latitude']
      DF.ix[i, 'Longitude'] = LL.iloc[j]['Longitude']
      break
'''

Postal code M3A coordinates are: latitude = 43.7532586; longitude = -79.3296565
Postal code M4A coordinates are: latitude = 43.725882299999995; longitude = -79.31557159999998
Postal code M5A coordinates are: latitude = 43.6542599; longitude = -79.3606359
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M6A coordinates are: latitude = 43.718517999999996; longitude = -79.46476329999999
Postal code M7A coordinates are: latitude = 43.6623015; longitude = -79.3894938
Postal code M9A coordinates are: latitude = 43.6678556; longitude = -79.53224240000002
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M1B coordinates are: latitude = 43.806686299999996; longitude = -79.19435340000001
Postal code M3B coordinates are: latitude = 43.745905799999996; longitude = -79.352188
Postal code M4B coordinates are: latitude = 43.7063972; longitude = -79.309937
Postal code M4B coordinates are:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  


Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M5B coordinates are: latitude = 43.6571618; longitude = -79.37893709999999
Postal code M6B coordinates are: latitude = 43.709577; longitude = -79.44507259999999
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M9B coordinates are: latitude = 43.6509432; longitude = -79.55472440000001
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M1C coordinates are: latitude = 43.7845351; longitude = -79.16049709999999
Postal code M3C coordinates are: latitude = 

In [0]:
DF.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.654260,-79.360636
3,M4A,North York,Victoria Village,43.718518,-79.464763
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763
5,M6A,North York,Lawrence Heights,43.662301,-79.389494
6,M6A,North York,Lawrence Manor,43.667856,-79.532242


# END OF PART 2

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

First it is necessary to drop all rows from which contains NaN values.

Then, in order to limit the analysis only for Toronto area, one neeeds to filter the rows keeping only the boroughs whose name contains "Toronto".

In [0]:
DF.dropna(inplace=True)
DF_toronto = DF[DF['Borough'].str.contains('Toronto')]

In [0]:
DF_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763
7,M7A,Downtown Toronto,Queen's Park,43.806686,-79.194353
16,M5B,Downtown Toronto,Ryerson,43.650943,-79.554724
17,M5B,Downtown Toronto,Garden District,43.650943,-79.554724
33,M5C,Downtown Toronto,St. James Town,43.763573,-79.188711
46,M4E,East Toronto,The Beaches,43.754328,-79.442259
47,M5E,Downtown Toronto,Berczy Park,43.705369,-79.349372
56,M5G,Downtown Toronto,Central Bay Street,43.778517,-79.346556
57,M6G,Downtown Toronto,Christie,43.767980,-79.487262
67,M5H,Downtown Toronto,Adelaide,43.727929,-79.262029


In order to visualize Toronto area within a interactive map, it requires the latitude and longitude coordinates. They were retrieved using the Nominatim package.

In [0]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Then a map in created using the Folium package.

In [0]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(DF_toronto['Latitude'], DF_toronto['Longitude'], DF_toronto['Borough'], DF_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In order to use the Foursquare API, it is necessary to input client ID, client secret, version and limit.

In [0]:
CLIENT_ID = '4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI' # your Foursquare ID
CLIENT_SECRET = 'UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4FJMQVQY0O2GDFFQLW3NBT20XONVI1SDAX2ODVF1M2XYPJEI
CLIENT_SECRET:UTVH1JPXDP3SC4OZUUQHHY2R5D0NWQS5EHMDWQTTB3KMWZ44


This function aims at listing interesting places that are located within 500 units radius from the point defined by some pair of latitude / longitude values.

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

This list is the result from using the function above.

In [0]:
toronto_venues = getNearbyVenues(names=DF_toronto['Neighbourhood'],
                                   latitudes=DF_toronto['Latitude'],
                                   longitudes=DF_toronto['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market


The resulting dataframe, which, besides the Neighborhood name, also contains latitude, longitude from both the neighborhoods themselves and the venues, including their category.

In [0]:
#toronto_venues.set_index(list(toronto_venues)[0], inplace=True)
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.718518,-79.464763,Roots,43.718221,-79.466776,Boutique
1,Harbourfront,43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),43.719096,-79.462675,Furniture / Home Store
2,Harbourfront,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant
3,Harbourfront,43.718518,-79.464763,Orfus Road Shopping Outlets,43.719045,-79.460849,Clothing Store
4,Harbourfront,43.718518,-79.464763,Tim Hortons,43.719427,-79.467995,Coffee Shop


The next step would be to format the above dataframe using the one hot encoding.

In [0]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot = pd.concat([toronto_onehot['Neighborhood'].to_frame(), toronto_onehot.drop(columns=['Neighborhood'], axis=1)], axis=1)
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,...,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


The problem is that this format is very sparse making the dimension of the dataframe to be very large.

In [0]:
toronto_onehot.shape

(874, 192)

One solution to this question would be to average all venues from the same neighborghood, resulting in the relative frequency of that category within that neighborhood.

In [0]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,...,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.0,...,0.000000,0.052632,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.052632,0.00000,0.000000,0.000000,0.000000,0.052632
2,Brockton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.0,...,0.000000,0.080000,0.000000,0.00,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.031746,0.015873,0.015873,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.00,0.015873,0.015873,0.00,0.015873,0.000000,0.000000,0.0,...,0.015873,0.015873,0.000000,0.00,0.015873,0.000000,0.000000,0.000000,0.015873,0.000000,0.0,0.000000,0.015873,0.015873,0.000000,0.00000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.031746,0.000000,0.015873,0.015873,0.000000,0.00000,0.015873,0.00000,0.000000,0.015873,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.031746,0.000000
4,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

... making the dataframe dimension to be reduced.

In [0]:
toronto_grouped.shape

(54, 192)

If, for each neighborhood, one sorts all venues by its frequency, it is possible to build a list of the most frequent venue category for each neighborhood.

In [0]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
               venue  freq
0        Coffee Shop  0.25
1         Hobby Shop  0.25
2     Discount Store  0.25
3   Department Store  0.25
4  Accessories Store  0.00


----Berczy Park----
               venue  freq
0  Indian Restaurant  0.11
1        Yoga Studio  0.05
2               Bank  0.05
3      Grocery Store  0.05
4        Gas Station  0.05


----Brockton----
                     venue  freq
0              Coffee Shop  0.08
1               Restaurant  0.08
2           Sandwich Place  0.08
3       Italian Restaurant  0.08
4  Comfort Food Restaurant  0.04


----Central Bay Street----
                  venue  freq
0        Clothing Store  0.14
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.06
3   Japanese Restaurant  0.05
4             Juice Bar  0.03


----Chinatown----
               venue  freq
0     Baseball Field   1.0
1  Accessories Store   0.0
2             Lounge   0.0
3     Massage Studio   0.0
4     Medical Center   0.0


----Christie----
          

Next define a function to list the most common venues for each row of the above dataframe.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Format the above dataframe to show, for each neighborhood, the most common venue category on its columns.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Discount Store,Hobby Shop,Department Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center
1,Berczy Park,Indian Restaurant,Yoga Studio,Bank,Grocery Store,Gym,Fast Food Restaurant,Intersection,Liquor Store,Coffee Shop,Park
2,Brockton,Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Sushi Restaurant,Thai Restaurant,Juice Bar,Spa,Cupcake Shop,Pub
3,Central Bay Street,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Bakery,Cosmetics Shop,Juice Bar,Tea Room,Women's Store,Spa
4,Chinatown,Baseball Field,Yoga Studio,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner


Cluster all neiborhood based on the 10 most common venue categories.

In [0]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 2, 1, 1], dtype=int32)

In [0]:
DF_toronto.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


Add to Toronto dataframe information about the resulting label, latitude and longitude.

In [0]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = DF_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,Harbourfront,43.718518,-79.464763,1,Furniture / Home Store,Women's Store,Clothing Store,Accessories Store,Coffee Shop,Gift Shop,Miscellaneous Shop,Boutique,Event Space,Vietnamese Restaurant
7,M7A,Downtown Toronto,Queen's Park,43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
16,M5B,Downtown Toronto,Ryerson,43.650943,-79.554724,3,Home Service,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
17,M5B,Downtown Toronto,Garden District,43.650943,-79.554724,3,Home Service,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Diner
33,M5C,Downtown Toronto,St. James Town,43.763573,-79.188711,1,Electronics Store,Rental Car Location,Breakfast Spot,Intersection,Medical Center,Mexican Restaurant,Spa,Coworking Space,Creperie,Drugstore


Finally create a map showing all neighborhood, color coded by cluster labels.

In [0]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged.dropna(inplace=True)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(cluster)-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Observations about clustering results:

1.   The are 5 different clusters
2.   Those cluster are very different in relation to their size
3. The larger one is number 1, which seems to be popular places
4. The smaller one is number 4, which seems to be places for particular kinds of interest.
5. The cluster number 1 spreads over almost all Toronto are. It is not very location specific.



# END OF PART 3 (IN ORDER TO SEE MAPS, ALL CELLS IN THE NOTEBOOK NEED TO BE EXECUTED)